In [20]:
import numpy as np
import pandas as pd
import FdsPy.spar.myspengine as sp
import FdsPy.qe.myqengine as qe

In [21]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M')

univ = qe.ScreenUniverse(universe_expr = '(ISAV(FFD_SHR_CLS_AUM(0,ACT,ROLL,USD))=1 AND ISAV(P_PRICE(0,USD))=1  AND UPPERCASE(P_EXCOUNTRY(ISO2))=UPPERCASE("US") AND PROPER(FFD_CLASS_ASSET(TEXT))=PROPER("EQUITY"))=1')
data_items = {'name':"FFD_NAME",
              'price':'P_PRICE(0,USD)',
              'aum':"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)",
              'lss_ticker':"LSS_TICKER",
              'ffd_bmk_id':"FFD_BMK_ID",
              'ffd_brand':"FFD_BRAND",
              'focus':'FFD_CLASS_FOCUS(TEXT)',
              'segment':'FFD_SEG',
              'niche':'FFD_CLASS_NICHE(TEXT)',
              'class':'FFD_CLASS_ASSET(TEXT)',
              'category':'FFD_CLASS_CAT(TEXT)'
              }

req = qe.Calculation(universe = univ, dates = time_series,data_dict= data_items)
req.query()


In [22]:
df = req.data
df.head()

,DATE,UNIVERSE,name,price,aum,lss_ticker,ffd_bmk_id,ffd_brand,focus,segment,niche,class,category
0,20230831,RMT,Royce Micro-Cap Trust Incorporated,8.980,5.010069e+08,RMT-US,R.2000,Royce,Micro Cap,Equity: U.S. - Micro Cap,Broad-based,Equity,Size and Style
1,20230831,FUNYX,Pioneer Fundamental Growth Fund,30.590,3.814054e+09,None,R.1000G,Pioneer,Large Cap,Equity: U.S. - Large Cap Growth,Growth,Equity,Size and Style
2,20230831,EMF,Templeton Emerging Markets Fund,11.705,2.161835e+08,EMF-US,NOINDEX,Templeton,Total Market,Equity: Emerging Markets - Total Market,Broad-based,Equity,Size and Style
3,20230831,ABCAX,American Aadvantage Funds,20.690,1.386211e+08,None,R.1000V,American Beacon,Total Market,Equity: U.S. - Total Market,Broad-based,Equity,Size and Style
4,20230831,TACWX,Templeton China World Fund,8.410,2.560751e+07,None,NOINDEX,Templeton,Total Market,Equity: China - Total Market,Broad-based,Equity,Size and Style


In [23]:
df=df.dropna().sort_values('aum',ascending = False).head(500)

In [24]:
import pandas as pd

#Spar inputs
doc_name = "PERSONAL:RETURNS_API"
comp_name = 'Cumulative Returns Table'
comp_cat = 'Raw Data / Returns'
bench_prefix = 'SPN'

ports = df['lss_ticker'].to_list()
bench = 'SP50'
ret_type = 'NTR'
freq = 'Daily'
curr = 'USD'
df_all = pd.DataFrame()

for horizon in ['0M','0Q','0Y']:
    startdate = horizon
    enddate = '0D'

    #post call to SPAR api
    spar_df = sp.calc_spar_data(doc_name,comp_name,comp_cat,ports,bench,bench_prefix,ret_type,startdate,enddate,freq,curr)

    df_full = pd.DataFrame()
    for port in list(spar_df.keys())[1:]:
        df_temp = spar_df[port][0]
        df_temp['ticker'] = port
        df_temp.iloc[:,1] = df_temp.iloc[:,1]
        df_temp =df_temp.rename(columns = {df_temp.columns[1]:port})
        df_temp= df_temp.set_index(['Dates'])
        df_full= pd.concat([df_full,df_temp[port]],axis=1)


    df_m = pd.DataFrame(df_full.stack().rename(f'cum_ret_{horizon}')).reset_index()
    df_m = df_m.rename(columns = {'level_0':'DATE',
                                'level_1':'lss_ticker'})
    df_m = df_m.set_index(['DATE','lss_ticker'])
    df_all = pd.concat([df_all,df_m],axis=1)


Calculation Unit Id:USA-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '0a8ed7fd8d8f4c1e80b8ab7012953aed:USA-US'}]
Calculation Unit Id:ZIPIMX-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '0a8ed7fd8d8f4c1e80b8ab7012953aed:ZIPIMX-US'}]
Calculation Unit Id:USA-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '18d54a91fc324c56a932d860d1fef041:USA-US'}]
Calculation Unit Id:ZIPIMX-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '18d54a91fc324c56a932d860d1fef041:ZIPIMX-US'}]
Calculation Unit Id:USA-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': 'b7c6ab246c834e13bf26d947aa241148:USA-US'}]
Calculation Unit Id:ZIPIMX-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': 'b7c6ab246c834e13bf26d947aa241148:ZIPIMX-US'}]


In [25]:

df = df.set_index(['lss_ticker'])
df_m = df_all.join(df,how='left')
df_m

cum_ret_0M  cum_ret_0Q cum_ret_0Y      DATE UNIVERSE  \
DATE       lss_ticker                                                         
08/31/2023 SWPPX-US           0.0    1.572737  18.703201  20230831    SWPPX   
           JLGMX-US           0.0    2.452076  26.449273  20230831    JLGMX   
           DFQTX-US           0.0    1.650172  14.290641  20230831    DFQTX   
           DFEOX-US           0.0    1.759630  15.407957  20230831    DFEOX   
           DFIEX-US           0.0    0.342904  10.130957  20230831    DFIEX   
...                           ...         ...        ...       ...      ...   
06/29/2023 REVIX-US           NaN         NaN   5.665047  20230831    REVIX   
           VVOSX-US           NaN         NaN    1.37671  20230831    VVOSX   
           BRXVX-US           NaN         NaN   8.256898  20230831    BRXVX   
           NEJYX-US           NaN         NaN  13.993802  20230831    NEJYX   
           ASG-US             NaN         NaN  11.938178  20230831      ASG   

                                                                    name  \
DATE       lss_ticker                                                      
08/31/2023 SWPPX-US                                 Schwab Capital Trust   
           JLGMX-US                                     JPMorgan Trust I   
           DFQTX-US                       DFA US Core Equity 2 Portfolio   
           DFEOX-US                       DFA US Core Equity 1 Portfolio   
           DFIEX-US              DFA International Core Equity Portfolio   
...                                                                  ...   
06/29/2023 REVIX-US    T. Rowe Price Emerging Markets Discovery Stock...   
           VVOSX-US                                     AIM Sector Funds   
           BRXVX-US                                  MFS Series Trust IV   
           NEJYX-US                                Natixis Funds Trust I   
           ASG-US                       Liberty All-Star Growth Fund Inc   

                       price           aum ffd_bmk_id       ffd_brand  \
DATE       lss_ticker                                                   
08/31/2023 SWPPX-US    69.75  7.462342e+10       SP50          Schwab   
           JLGMX-US    59.33  4.126916e+10    R.1000G        JPMorgan   
           DFQTX-US    30.80  3.097797e+10     R.3000             DFA   
           DFEOX-US    34.12  3.047367e+10     R.3000             DFA   
           DFIEX-US    14.63  2.985577e+10     991000             DFA   
...                      ...           ...        ...             ...   
06/29/2023 REVIX-US    12.94  3.393625e+08    NOINDEX   T. Rowe Price   
           VVOSX-US    17.26  3.375187e+08      SPSUP         Invesco   
           BRXVX-US    11.82  3.374265e+08     899901             MFS   
           NEJYX-US    18.81  3.300355e+08    R.2000V         Natixis   
           ASG-US       5.29  3.273305e+08    NOINDEX  Liberty Street   

                              focus  \
DATE       lss_ticker                 
08/31/2023 SWPPX-US       Large Cap   
           JLGMX-US       Large Cap   
           DFQTX-US    Total Market   
           DFEOX-US    Total Market   
           DFIEX-US    Total Market   
...                             ...   
06/29/2023 REVIX-US    Total Market   
           VVOSX-US    Total Market   
           BRXVX-US    Total Market   
           NEJYX-US       Small Cap   
           ASG-US      Total Market   

                                                         segment        niche  \
DATE       lss_ticker                                                           
08/31/2023 SWPPX-US                   Equity: U.S.  -  Large Cap  Broad-based   
           JLGMX-US           Equity: Global  - Large Cap Growth       Growth   
           DFQTX-US                Equity: U.S.  -  Total Market  Broad-based   
           DFEOX-US                Equity: U.S.  -  Total Market  Broad-based   
           DFIEX-US      Equity: Global Ex-U.S.  -  Total Mark

In [26]:
df_m.to_pickle('../../data/Performance Leaders/univ_cumulative_returns_multihorizon_500_20230918.pkl')